<a href="https://colab.research.google.com/github/SophiaU/Git_DoD_Remittance/blob/main/New_Daily_Data_Dump_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Package Installations

In [ ]:
# install and import pygsheets
!pip install pygsheets
import pygsheets
import gspread

# import pandas
import pandas as pd
from functools import reduce

#  Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


     |████████████████████████████████| 147 kB 7.0 MB/s 


### Authentications

##### Import The JSON File

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving service_account.json to service_account.json


In [ ]:
#  Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# authenticate gspread
gc = gspread.authorize(GoogleCredentials.get_application_default())

#authenticate pygsheets
import json
from google.oauth2 import service_account

with open('service_account.json') as source:
   info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

client = pygsheets.authorize(service_account_file='service_account.json')

### Global Variables

In [ ]:
day = 'Nov 18'

weekFolderID = '1BcYdBsK5UQYD6YUD5eOo5-8w_LP3D86l'
weekFolder = drive.ListFile({'q': "'"+weekFolderID+"' in parents and  mimeType = 'application/vnd.google-apps.folder'"}).GetList()
VAFolderID = '1cDvmmJBVmz4FPpePljiZ1clEMnbo9LPQ'

lagos = "(not title contains '(12)' and not title contains '(13)' and not title contains '(14)' and not title contains '(30)' and not title contains '(20)' and not title contains '(10)' and not title contains '(9)' and not title contains '(7)' and not title contains '(6)')"
dataDumpLagos = '1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg'

ghana = "(title contains '(6)' or title contains '(9)')"
dataDumpGhana = '1bfBtlRbueriORkEYE4jXv4yHmFsOr_tdYZlGlVp8cJ0'

sa = "(title contains '(7)' or title contains '(10)' or title contains '(30)' or title contains '(20)')"
dataDumpSA = '1YDi41F705U9R4MBda0tILrkxR0uM8b9iX-MK5yombQQ'

pd.set_option('display.max_columns', 500)


Combine Driver Performance Files

In [ ]:
def combine_performance_files(country):
  # Append all files in folder into a listframe

  listframe = []


  for folder in weekFolder:
    if folder['title'] == 'Performance':
      # https://developers.google.com/drive/v2/web/search-parameters
      files = drive.ListFile({'q': "title contains '"+day+"'and "+country+" and '"+folder['id']+"' in parents \
                              and mimeType = 'text/csv' and trashed = false "}).GetList()
      for file in files:
        print(file['title'])
        # download
        fileDownloaded = drive.CreateFile({'id':file['id']})

        # load
        fileDownloaded.GetContentFile('performance.csv')

        # read
        df = pd.read_csv('performance.csv', delimiter=',' )

        #Renaming
        df = df.rename({'Time on trip (days: hours : minutes)':'Time on trip (days : hours : minutes)','Driver last name':'Driver surname'}, axis=1)

        # append
        listframe.append(df)

  # combine all files in the list into a dataframe
  dayperffile = pd.concat(listframe)

# Driver UUID	Driver first name	Driver surname	Trips completed	Time online (days : hours: minutes)	Time on trip (days : hours : minutes)

  # convert hour online to just hours
  dayperffile['Time online (days : hours: minutes)'] = round(pd.to_timedelta(dayperffile['Time online (days : hours: minutes)']).dt.total_seconds()/60,2)
  dayperffile['Time on trip (days : hours : minutes)'] = round(pd.to_timedelta(dayperffile['Time on trip (days : hours : minutes)']).dt.total_seconds()/60,2)

  # convert strings to numerics
  columnslist = ['Trips completed', 'Time online (days : hours: minutes)', 'Time on trip (days : hours : minutes)']
  for a in columnslist:
    dayperffile[a] = pd.to_numeric(dayperffile[a],errors='coerce')

  #Rename
  dayperffile = dayperffile.rename({'Trips completed':'PerfTrips completed'}, axis=1)

  # Combine first and last names
  dayperffile['Name'] = dayperffile['Driver first name'] + " " + dayperffile['Driver surname']

  # aggregate metrics by Driver UUID
  dayperffile = dayperffile.groupby(['Driver UUID','Name']).agg({'PerfTrips completed':'sum','Time online (days : hours: minutes)':'sum', 'Time on trip (days : hours : minutes)':'sum'}).reset_index()

  # print(dayperffile)
  return dayperffile


Combine Driver Quality Files

In [ ]:
def combine_driverquality_files(country):
  # Append all files in folder into a listframe

  listframe = []


  for folder in weekFolder:
    if folder['title'] == 'Driver Quality':
      # https://developers.google.com/drive/v2/web/search-parameters
      files = drive.ListFile({'q': "title contains '"+day+"'and "+country+" and '"+folder['id']+"' in parents \
                              and mimeType = 'text/csv' and trashed = false "}).GetList()
      for file in files:
        print(file['title'])
        # download
        fileDownloaded = drive.CreateFile({'id':file['id']})

        # load
        fileDownloaded.GetContentFile('driver_quality.csv')

        # read
        df = pd.read_csv('driver_quality.csv', delimiter=',' )

        # Rename Acceptance rate to Confirmation rate
        df = df.rename({'Acceptance rate':'Confirmation rate','Driver last name':'Driver surname'}, axis=1)

        # append
        listframe.append(df)

  # combine all files in the list into a dataframe
  daydrvqualityfile = pd.concat(listframe)

  
# Driver UUID	Driver first name	Driver surname	Trips completed	Confirmation rate	Cancellation rate	Completion rate	Driver ratings (last 4 weeks)	Driver ratings (previous 500 trips)

  # convert strings to numerics
  columnslist = ['Trips completed', 'Confirmation rate', 'Cancellation rate', 'Completion rate', 'Driver ratings (last 4 weeks)', 'Driver ratings (previous 500 trips)']
  for a in columnslist:
    daydrvqualityfile[a] = pd.to_numeric(daydrvqualityfile[a],errors='coerce')

  tripcompleted =daydrvqualityfile['Trips completed'].astype(str).str.split(" ",n=0,expand=True)
  daydrvqualityfile['Trips completed'] = tripcompleted[0].fillna(0)
  daydrvqualityfile['Trips completed'] = pd.to_numeric(daydrvqualityfile['Trips completed'])

  confirmationrate =daydrvqualityfile['Confirmation rate'].astype(str).str.split(" ",n=0,expand=True)
  daydrvqualityfile['Confirmation rate'] = confirmationrate[0].fillna(0)
  daydrvqualityfile['Confirmation rate'] = pd.to_numeric(daydrvqualityfile['Confirmation rate'],errors='coerce')

  cancellationrate =daydrvqualityfile['Cancellation rate'].astype(str).str.split(" ",n=0,expand=True)
  daydrvqualityfile['Cancellation rate'] = cancellationrate[0].fillna(0)
  daydrvqualityfile['Cancellation rate'] = pd.to_numeric(daydrvqualityfile['Cancellation rate'],errors='coerce')

  completionrate =daydrvqualityfile['Completion rate'].astype(str).str.split(" ",n=0,expand=True)
  daydrvqualityfile['Completion rate'] = cancellationrate[0].fillna(0)
  daydrvqualityfile['Completion rate'] = pd.to_numeric(daydrvqualityfile['Completion rate'],errors='coerce')

  daydrvqualityfile['Driver ratings (last 4 weeks)'] = pd.to_numeric(daydrvqualityfile['Driver ratings (last 4 weeks)'],errors='coerce')
  daydrvqualityfile['Driver ratings (previous 500 trips)'] = pd.to_numeric(daydrvqualityfile['Driver ratings (previous 500 trips)'],errors='coerce')

  # aggregate metrics by Uber ID
  daydrvqualityfile = daydrvqualityfile.groupby('Driver UUID').agg({'Trips completed':'sum', 'Confirmation rate':'mean','Cancellation rate':'mean', 'Completion rate':'mean',\
                                              'Driver ratings (last 4 weeks)':'mean', 'Driver ratings (previous 500 trips)':'mean'}).reset_index()

  # print(daydrvqualityfile)
  return daydrvqualityfile


Combine Trip Activity

In [ ]:
def combine_tripactivity_files(country):
  # Append all files in folder into a listframe

  listframe = []


  for folder in weekFolder:
    if folder['title'] == 'Trip Activity':
      # https://developers.google.com/drive/v2/web/search-parameters
      files = drive.ListFile({'q': "title contains '"+day+"'and "+country+" and '"+folder['id']+"' in parents \
                              and mimeType = 'text/csv' and trashed = false "}).GetList()
      for file in files:
        print(file['title'])
        # download
        fileDownloaded = drive.CreateFile({'id':file['id']})

        # load
        fileDownloaded.GetContentFile('trip_activity.csv')

        # read
        df = pd.read_csv('trip_activity.csv', delimiter=',' )

        # append
        listframe.append(df)

  # combine all files in the list into a dataframe
  combinedtripactivityfile = pd.concat(listframe)

  # convert strings to numerics
  combinedtripactivityfile['Trip distance'] = pd.to_numeric(combinedtripactivityfile['Trip distance'],errors='coerce')

  tripDistance =combinedtripactivityfile['Trip distance'].astype(str).str.split(" ",n=0,expand=True)
  combinedtripactivityfile['Trip distance'] = tripDistance[0].fillna(0)
  combinedtripactivityfile['Trip distance'] = pd.to_numeric(combinedtripactivityfile['Trip distance'])

  # pick only Uber ID and Trips Distance
  daytripactivityfile =combinedtripactivityfile[['Driver UUID', 'Trip distance']]
  # return daytripactivityfile

  # get total trip_distance
  # sum trip distance by Driver UUID
  trip_distance = daytripactivityfile.groupby('Driver UUID').agg({'Trip distance':'sum'}).reset_index()
  # trip_distance.columns = ['Driver UUID', 'Trip distance']

  # print(trip_distance)

  return trip_distance

# Trip UUID	Driver UUID	Driver first name	Driver surname	Vehicle UUID	Number plate	Service type	Trip request time	Trip drop-off time	Pick-up address	Drop-off address	Trip distance	Trip status



Combine Payments

In [ ]:
def combine_payments_files(country):
  # Append all files in folder into a listframe

  listframe = []


  for folder in weekFolder:
    if folder['title'] == 'Payments':
      # https://developers.google.com/drive/v2/web/search-parameters
      files = drive.ListFile({'q': "title contains '"+day+"'and "+country+" and '"+folder['id']+"' in parents \
                              and mimeType = 'text/csv' and trashed = false "}).GetList()
      for file in files:
        print(file['title'])
        # download
        fileDownloaded = drive.CreateFile({'id':file['id']})

        # load
        fileDownloaded.GetContentFile('payments.csv')

        # read
        df = pd.read_csv('payments.csv', delimiter=',' )
        df = df.rename({'Paid to you : Your earnings':'Total Earnings','Paid to you : Trip balance : Payouts : Cash collected':'Cash Collected',
                        'Paid to you':'Uber Balance','Paid to you : Trip balance : Payouts : Cash Collected':'Cash Collected',\
                        'Paid to you : Your earnings : Fare':'Net Fares', 'Paid to you:Your earnings:Service fee':'Uber Fee',\
                        'Paid to you:Your earnings:Service Fee':'Uber Fee'}, axis=1)

        # append
        listframe.append(df)

  # combine all files in the list into a dataframe
  daypaymentsfile = pd.concat(listframe)

# transaction UUID	Driver UUID	Driver first name	Driver surname	Trip UUID	Description	Organisation name	Org alias	vs reporting	Paid to you	Paid to you : Your earnings	Paid to you : Trip balance : Payouts : Cash collected	Paid to you : Your earnings : Fare	Paid to you : Your earnings : Taxes	Paid to you:Your earnings:Fare:Fare	Paid to you:Your earnings:Fare:Surge	Paid to you:Your earnings:Service fee	Paid to you:Your earnings:Fare:Wait Time at Pick-up	Paid to you:Your earnings:You saved:Quest	Paid to you:Your earnings:Fare:Cancellation	Paid to you:Trip balance:Refunds:Airport fee	Paid to you:Your earnings:Tip	Paid to you:Trip balance:Refunds:Toll	Paid to you:Trip balance:Payouts:Transferred To Bank Account	Paid to you:Your earnings:Fare:Adjustment

  # aggregate metrics by Driver UUID
  daypaymentsfile = daypaymentsfile.groupby('Driver UUID').agg({'Total Earnings':'sum','Cash Collected':['sum', lambda x: (x < 0).sum()],'Uber Balance':'sum','Net Fares':'sum','Uber Fee':'sum' }).reset_index()
  daypaymentsfile.columns = ['Driver UUID','Total Earnings','CashCollected','CashTrips', 'Uber Balance','Net Fares', 'Uber Fee']


  # print(daypaymentsfile)
  return daypaymentsfile



Get Moove Details

In [ ]:
def moove_details(dumpsheet):
  # get moove details
  mooveDetailList =[]
  worksheet = gc.open_by_key(dumpsheet).sheet1
  for i in range(2,6):
    columns = worksheet.col_values(i)
    mooveDetailList.append(columns[1:])

  # print(mooveDetailList)

  # convert list to data frame
  mooveDetails = pd.DataFrame(mooveDetailList)
  mooveDetails = mooveDetails.T
  mooveDetails.columns = ['Name','Product','DRN', 'Driver UUID']
  mooveDetails['Name'] = pd.Series(mooveDetails['Name']).astype(str).str.title()

  # print(mooveDetails)
  return mooveDetails

#### Combine All Files and Moove Details

In [ ]:
def generate_reconfile(driverquality_file,perffile,tripactivityfile,paymentsfile,mooveDetails):

  # only unique driver names and uber IDs from Moove details
  # mooveDetailsName = mooveDetails[['Driver UUID', 'Name']].drop_duplicates(subset='Name')

  # join on driverUUID
  # mooveDetailsID = mooveDetails[['Product','DRN', 'Driver UUID']].drop_duplicates(subset='Driver UUID')

  dfs = [driverquality_file,perffile,tripactivityfile,paymentsfile]
  reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='Driver UUID'), dfs)

  # wrangle reconfile
  reconfile = reconfile.fillna(0)

  reconfile['Date'] = day
  reconfile['CashCollected'] = -reconfile['CashCollected']
  reconfile['GrossFares'] = reconfile['Net Fares'] - reconfile['Uber Fee']
  reconfile['GrossFT'] = round(reconfile['GrossFares']/reconfile['PerfTrips completed'],2)
  reconfile['TripsAccepted'] = round(reconfile['PerfTrips completed']/(1-reconfile['Cancellation rate']),0)
  reconfile['CardTrips'] = reconfile['PerfTrips completed']-reconfile['CashTrips']
  reconfile['CardRevenue'] = reconfile['GrossFares'] - reconfile['CashCollected']

  reconfile = reconfile[['Date','Driver UUID', 'Name','PerfTrips completed', 'Time online (days : hours: minutes)', 'Time on trip (days : hours : minutes)', 
                         'Confirmation rate', 'Cancellation rate', 'Completion rate', 'Driver ratings (last 4 weeks)', 'Driver ratings (previous 500 trips)',
                         'Trip distance', 'Total Earnings', 'CashCollected', 'Uber Balance','Net Fares', 'Uber Fee', 
                         'GrossFares', 'GrossFT', 'TripsAccepted', 'CashTrips', 'CardTrips', 'CardRevenue']]
  reconfile.drop_duplicates(subset=['Name', 'Driver UUID','PerfTrips completed'],inplace=True)

  # reconfile.head(400)

  return reconfile


Write Data to Google Sheets

In [ ]:
def write_to_sheets(reconfile,dumpsheet):
  reconSheet = client.open_by_key(dumpsheet) 
  wksalldriversfile = reconSheet.worksheet_by_title("newDailyData")
  lastrow = len(wksalldriversfile.get_values('A1','A40000'))+1
    
  wksalldriversfile.set_dataframe(reconfile,(lastrow,1),copy_head=False,nan='')

Lagos Data Dump

In [ ]:
def lagos_data_dump(country,dumpsheet):
  driverquality_file = combine_driverquality_files(country)
  perffile = combine_performance_files(country)
  tripactivityfile = combine_tripactivity_files(country)
  paymentsfile = combine_payments_files(country)
  mooveDetails = moove_details(dumpsheet)

  reconfile = generate_reconfile(driverquality_file,perffile,tripactivityfile,paymentsfile,mooveDetails)
  write_to_sheets(reconfile,dumpsheet)



Ghana Data Dump

In [ ]:
def ghana_data_dump(country,dumpsheet):
  driverquality_file = combine_driverquality_files(country)
  perffile = combine_performance_files(country)
  tripactivityfile = combine_tripactivity_files(country)
  paymentsfile = combine_payments_files(country)
  mooveDetails = moove_details(dumpsheet)

  reconfile = generate_reconfile(driverquality_file,perffile,tripactivityfile,paymentsfile,mooveDetails)
  write_to_sheets(reconfile,dumpsheet)

SA Data Dump

In [ ]:
def sa_data_dump(country,dumpsheet):
  driverquality_file = combine_driverquality_files(country)
  perffile = combine_performance_files(country)
  tripactivityfile = combine_tripactivity_files(country)
  paymentsfile = combine_payments_files(country)
  mooveDetails = moove_details(dumpsheet)

  reconfile = generate_reconfile(driverquality_file,perffile,tripactivityfile,paymentsfile,mooveDetails)
  write_to_sheets(reconfile,dumpsheet)
  

Run Data Dump

In [ ]:
# lagos_data_dump(lagos,dataDumpLagos)
# ghana_data_dump(ghana,dataDumpGhana)
sa_data_dump(sa,dataDumpSA)

driver_quality Nov 18(20).csv
driver_quality Nov 18(30).csv
driver_quality Nov 18(10).csv
driver_quality Nov 18(7).csv
driver_performance Nov 18(20).csv
driver_performance Nov 18(30).csv
driver_performance Nov 18(10).csv
driver_performance Nov 18(7).csv
trip_activity Nov 18(20).csv
trip_activity Nov 18(30).csv
trip_activity Nov 18(10).csv
trip_activity Nov 18(7).csv
Payments Nov 18(20).csv
Payments Nov 18(30).csv
Payments Nov 18(10).csv
Payments Nov 18(7).csv
